In [27]:
#Run 1st
#import libaries
#import libaries
import requests
import pandas as pd
import time
import psycopg2 as ps
from sodapy import Socrata

In [28]:
#Run 2nd
client = Socrata("data.cdc.gov", None)

results = client.get("8pt5-q6wp", limit=20000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
#Now its time to Connect to our database

In [29]:
def connect_to_db(host_name, dbname, port, username, password):
#We will use the connect method from the ps module to connect to our database
    try:
        conn = ps.connect(host = host_name, database = dbname, user = username, password = password, port = port)
    except ps.OperationalError as e:
        raise e
    else: 
        print('Connected!')
    return conn

In [ ]:
#Checks to ensure we are connected, none indicates we are not
print(type(conn))


In [ ]:
#Create table

def create_table(curr):
    create_table_command = """CREATE TABLE IF NOT EXISTS depress (
        indicator VARCHAR(255),
        "group" VARCHAR(50),
        state VARCHAR(50),
        subgroup VARCHAR(50),
        phase VARCHAR(50),  -- Adjusted to VARCHAR
        time_period VARCHAR(50),
        time_period_label VARCHAR(100),
        time_period_start_date DATE,
        time_period_end_date DATE,
        value FLOAT,
        lowci FLOAT,
        highci FLOAT,
        confidence_interval VARCHAR(40),
        quartile_range VARCHAR(40)
    )"""
    curr.execute(create_table_command)



In [ ]:
#alter table, add column
alter_table_command = """ALTER TABLE depress ADD COLUMN id SERIAL PRIMARY KEY"""
curr.execute(alter_table_command)

In [22]:
def add_id_column(curr):
    # Create a temporary table with the desired column order
    create_temp_table_command = """CREATE TABLE temp_depress (
        id SERIAL PRIMARY KEY,
        indicator VARCHAR(255),
        "group" VARCHAR(50),
        state VARCHAR(50),
        subgroup VARCHAR(50),
        phase VARCHAR(50),
        time_period VARCHAR(50),
        time_period_label VARCHAR(100),
        time_period_start_date DATE,
        time_period_end_date DATE,
        value FLOAT,
        lowci FLOAT,
        highci FLOAT,
        confidence_interval VARCHAR(40),
        quartile_range VARCHAR(40)
    )"""
    curr.execute(create_temp_table_command)

    # Copy data from the existing table to the temporary table
    copy_data_command = "INSERT INTO temp_depress (id, indicator, \"group\", state, subgroup, phase, time_period, time_period_label, time_period_start_date, time_period_end_date, value, lowci, highci, confidence_interval, quartile_range) SELECT id::INTEGER, indicator, \"group\", state, subgroup, phase, time_period, time_period_label, time_period_start_date, time_period_end_date, value, lowci, highci, confidence_interval, quartile_range FROM depress"
    curr.execute(copy_data_command)

    # Drop the existing table
    drop_table_command = "DROP TABLE depress"
    curr.execute(drop_table_command)

    # Rename the temporary table to the original table name
    rename_table_command = "ALTER TABLE temp_depress RENAME TO depress"
    curr.execute(rename_table_command)


In [ ]:
data = results_df.values.tolist()

def insert_data(curr, data):
    insert_command = """
    INSERT INTO depress (
        indicator, "group", state, subgroup, phase, time_period,
        time_period_label, time_period_start_date, time_period_end_date,
        value, lowci, highci, confidence_interval, quartile_range
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    curr.executemany(insert_command, data)

In [ ]:
#Use to see all columns in dataframe
print(results_df.columns)


In [30]:
host_name = 'localhost'
dbname = 'Healthcare'
port = xxxx
username = 'postgres'
password = 'xxxx'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [71]:
#update NaN in table
def update_table(curr):
    try:
        # Update the NaN values and non-numeric values in the quartile_range column
        update_quartile_range_command = """UPDATE depress
            SET quartile_range = 'N/A'
            WHERE quartile_range = 'NaN' OR quartile_range ~ '[^0-9.]'"""
        curr.execute(update_quartile_range_command)

        # Update the NaN values in the value, lowci, highci, and confidence_interval columns
        update_nan_values_command = """UPDATE depress
            SET value = NULL
            WHERE value = 'NaN';
            
            UPDATE depress
            SET lowci = NULL
            WHERE lowci = 'NaN';
            
            UPDATE depress
            SET highci = NULL
            WHERE highci = 'NaN';
            
            UPDATE depress
            SET confidence_interval = NULL
            WHERE confidence_interval = 'NaN';"""
        curr.execute(update_nan_values_command)

        # Commit the changes
        curr.execute("COMMIT")
        print("Table updated successfully.")
    except Exception as e:
        # Rollback the transaction if an error occurs
        curr.execute("ROLLBACK")
        raise e


In [73]:
update_table(curr)

Table updated successfully.


In [43]:
#allows you to run sql commands
curr = conn.cursor()

In [ ]:
create_table(curr)

In [ ]:
# Insert the data into the table
insert_data(curr, data)

In [24]:
#add column
add_id_column(curr)

In [74]:
#After est a connection and create table, need to commit to have changes go into db
conn.commit()

In [26]:
conn.close()